In [404]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch

In [405]:
inputs = pd.read_csv("iris.csv")
inputs.pop("Id")

targets = inputs.pop("Species")

inputs = np.array(inputs)
targets = np.array(targets)

new_targets = []

for _ in range(len(targets)):
    if targets[_]=="Iris-setosa":
        new_targets.append([1,0,0])
    elif targets[_]=="Iris-versicolor":
        new_targets.append([0,1,0])
    else:
        new_targets.append([0,0,1])
        
targets = np.array(new_targets)

print(targets.shape)


(150, 3)


In [406]:
inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs, targets, test_size= 0.33)

print(inputs_train.shape, targets_train.shape)


(100, 4) (100, 3)


In [407]:
inputs_train = StandardScaler().fit_transform(inputs_train)
inputs_test = StandardScaler().fit_transform(inputs_test)

In [408]:
inputs_train = torch.from_numpy(inputs_train.astype(np.float32)) 
inputs_test = torch.from_numpy(inputs_test.astype(np.float32)) 
targets_train = torch.from_numpy(targets_train.astype(np.float32)) 
targets_test = torch.from_numpy(targets_test.astype(np.float32)) 

In [409]:
class IrisModel(torch.nn.Module):
    def __init__(self,input_dim, output_dim):
        super().__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
    
    def forward(self, inputs):
        targets_pred = torch.softmax(self.linear(inputs),dim=1) 
        return targets_pred
        

In [410]:
#pipeline

logistic_model = IrisModel(4,3)

loss_fn = torch.nn.BCELoss()

opt = torch.optim.SGD(logistic_model.parameters(), lr=1e-2)

In [411]:
epochs = 1000

for epoch in range(epochs):
    targets_pred = logistic_model(inputs_train)
    
    loss = loss_fn(targets_pred, targets_train)
    
    loss.backward()
    
    opt.step()
    opt.zero_grad()
    
    if (epoch+1)%10 == 0:
        print("Epoch: ", epoch+1, "  Loss: ", loss.item())
        
        with torch.no_grad():
            targets_test_pred = logistic_model(inputs_test)
            targets_test_cls = targets_test_pred.detach().clone()
            for i in range(len(targets_test_cls)):
                for j in range(len(targets_test_cls[i])):
                    if targets_test_cls[i][j] == max(targets_test_cls[i]):
                        targets_test_cls[i][j] = 1
                    else:
                        targets_test_cls[i][j] = 0
            
            targets_test_temp = targets_test.detach().clone()
            
        
            
            correct = 0.0
            
            for i in range(len(targets_test_temp)):
                if targets_test_temp[i].eq(targets_test_cls[i]).sum()==3:
                    correct += 1
            
            acc = correct/len(targets_test)
            
            print(acc)                        
                               
            
            
            
            

Epoch:  10   Loss:  0.5604709982872009
0.24
Epoch:  20   Loss:  0.5411829352378845
0.26
Epoch:  30   Loss:  0.5233309864997864
0.28
Epoch:  40   Loss:  0.5067867636680603
0.3
Epoch:  50   Loss:  0.4914374351501465
0.38
Epoch:  60   Loss:  0.477183073759079
0.46
Epoch:  70   Loss:  0.4639344811439514
0.48
Epoch:  80   Loss:  0.45161107182502747
0.48
Epoch:  90   Loss:  0.4401394724845886
0.5
Epoch:  100   Loss:  0.4294525682926178
0.52
Epoch:  110   Loss:  0.41948869824409485
0.52
Epoch:  120   Loss:  0.41019099950790405
0.52
Epoch:  130   Loss:  0.40150681138038635
0.52
Epoch:  140   Loss:  0.39338764548301697
0.52
Epoch:  150   Loss:  0.3857887089252472
0.52
Epoch:  160   Loss:  0.3786686062812805
0.52
Epoch:  170   Loss:  0.37198925018310547
0.52
Epoch:  180   Loss:  0.3657155930995941
0.52
Epoch:  190   Loss:  0.3598153293132782
0.54
Epoch:  200   Loss:  0.3542589843273163
0.54
Epoch:  210   Loss:  0.34901946783065796
0.54
Epoch:  220   Loss:  0.3440718650817871
0.54
Epoch:  230   L